In [12]:
import pandas as pd #csv 파일은 pandas로 바로 부를 수 있음
import random
import matplotlib.pyplot as plt #예측 결과 차트로 그리기
from sklearn.preprocessing import MinMaxScaler 


import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [19]:
! pip install yfinance

     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     ------------------------------------- 948.2/948.2 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=139064 sha256=f871c7415e5b0e17ae28882c11c0eb735290e0f176848a842dadc3569ab8e07b
  Stored in directory: c:\users\dhdms\appdata\local\pip\cache\wheels\8f\65\34\456800445efeafb05164fe95285c70e81ba1d96bae30f43917
Successfully built peewee


In [20]:
import yfinance as yf

In [22]:
ticker = "005930.KS"
data = yf.download(ticker, start='2000-01-01', end='2024-12-30', interval='id')
data

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['005930.KS']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (id 2000-01-01 -> 2024-12-30) (Yahoo error = "Invalid input - interval=id is not supported. Valid intervals: , 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]")')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,005930.KS,005930.KS,005930.KS,005930.KS,005930.KS,005930.KS
Date,,,,,,


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 0 entries
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   (Adj Close, 005930.KS)  0 non-null      float64
 1   (Close, 005930.KS)      0 non-null      float64
 2   (High, 005930.KS)       0 non-null      float64
 3   (Low, 005930.KS)        0 non-null      float64
 4   (Open, 005930.KS)       0 non-null      float64
 5   (Volume, 005930.KS)     0 non-null      float64
dtypes: float64(6)
memory usage: 0.0 bytes


In [24]:
data.to_csv("005930.KS01.csv")

In [25]:
print(data.shape)

(0, 6)


Volume : 거래량

수정종가를 예측할 것임

In [13]:
df = pd.read_csv("005930.KS.csv")
print(df.head())
print(df.tail())

         Date     Open     High      Low    Close     Adj Close    Volume
0  2010-01-04  16060.0  16180.0  16000.0  16180.0  12385.366211  11950800
1  2010-01-05  16520.0  16580.0  16300.0  16440.0  12584.391602  27925850
2  2010-01-06  16580.0  16820.0  16520.0  16820.0  12875.269531  22948850
3  2010-01-07  16820.0  16820.0  16260.0  16260.0  12446.605469  22107950
4  2010-01-08  16400.0  16420.0  16120.0  16420.0  12569.078125  14777550
            Date     Open     High      Low    Close  Adj Close    Volume
3499  2024-03-14  74400.0  74500.0  73600.0  74300.0    74300.0  22545539
3500  2024-03-15  73400.0  73700.0  72300.0  72300.0    72300.0  22580555
3501  2024-03-18  72600.0  73000.0  72500.0  72800.0    72800.0  11520348
3502  2024-03-19  72300.0  73000.0  71700.0  72800.0    72800.0  15376066
3503  2024-03-20  73700.0  77200.0  73400.0  76900.0    76900.0  50106297


In [14]:
df.info()
# df = df.set_index()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3504 non-null   object 
 1   Open       3504 non-null   float64
 2   High       3504 non-null   float64
 3   Low        3504 non-null   float64
 4   Close      3504 non-null   float64
 5   Adj Close  3504 non-null   float64
 6   Volume     3504 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 191.8+ KB


In [15]:
df['Date']

0       2010-01-04
1       2010-01-05
2       2010-01-06
3       2010-01-07
4       2010-01-08
           ...    
3499    2024-03-14
3500    2024-03-15
3501    2024-03-18
3502    2024-03-19
3503    2024-03-20
Name: Date, Length: 3504, dtype: object

In [16]:
df =df.set_index(pd.to_datetime(df.Date))
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2010-01-04,2010-01-04,16060.0,16180.0,16000.0,16180.0,12385.366211,11950800
2010-01-05,2010-01-05,16520.0,16580.0,16300.0,16440.0,12584.391602,27925850
2010-01-06,2010-01-06,16580.0,16820.0,16520.0,16820.0,12875.269531,22948850
2010-01-07,2010-01-07,16820.0,16820.0,16260.0,16260.0,12446.605469,22107950
2010-01-08,2010-01-08,16400.0,16420.0,16120.0,16420.0,12569.078125,14777550


수정종가를 얻기 위한 데이터 작업

In [18]:
df = df[['Open', 'High', 'Low', 'Close', 'Volume','Adj Close']]
df.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,16060.0,16180.0,16000.0,16180.0,11950800,12385.366211
2010-01-05,16520.0,16580.0,16300.0,16440.0,27925850,12584.391602
2010-01-06,16580.0,16820.0,16520.0,16820.0,22948850,12875.269531
2010-01-07,16820.0,16820.0,16260.0,16260.0,22107950,12446.605469
2010-01-08,16400.0,16420.0,16120.0,16420.0,14777550,12569.078125
